# Run a Documentation Template

This notebook shows how to run all tests defined in a documentation template.

## Before you begin

To use the ValidMind Developer Framework with a Jupyter notebook, you need to install and initialize the client library first, along with getting your Python environment ready.

If you don't already have one, you should also [create a documentation project](https://docs.validmind.ai/guide/create-your-first-documentation-project.html) on the ValidMind platform. You will use this project to upload your documentation and test results.

## Install the client library

In [ ]:
%pip install --upgrade validmind

## Initialize the client library

In a browser, go to the **Client Integration** page of your documentation project and click **Copy to clipboard** next to the code snippet. This code snippet gives you the API key, API secret, and project identifier to link your notebook to your documentation project.

::: {.column-margin}
::: {.callout-tip}
This step requires a documentation project. [Learn how you can create one](https://docs.validmind.ai/guide/create-your-first-documentation-project.html).
:::
:::

Next, replace this placeholder with your own code snippet:

In [ ]:
## Replace with code snippet from your documentation project ##

import validmind as vm

vm.init(
    api_host="https://api.prod.validmind.ai/api/v1/tracking",
    api_key="...",
    api_secret="...",
    project="..."
)

## Structure of a documentation template

All projects created in ValidMind are based on a documentation template. A documentation template is a collection of content blocks that, when rendered, produce a document that model developers can use for model validation.

The template structure is a simple combination of content `sections` where each section can have one or more content blocks. We currently support text-based content blocks (that are populated with the ValidMind UI) and test-driven content blocks. Under the hood, a template is represented as a YAML file. For more information about a template's structure, please refer to [this page](https://docs.validmind.ai/guide/edit-templates.html).

## Previewing a template

We can use the `preview_template()` function to preview the content blocks that will be populated by the developer framework.

In [ ]:
vm.preview_template()

## Running a template

We can use the `run_template()` function to run all tests defined in a documentation template. Note that each test in the template will have specific `context` requirements. Each context object can be passed as a function argument. For example, if a test requires a `model` object, we can pass it as a function argument:

```python
vm.run_template(model=vm_model) # vm_model is a validmind Model instance
```

Let's run through our customer churn demo to illustrate how to run a template and pass a model and dataset context to the `run_template()` function.

In [ ]:
# Load a demo dataset and train a simple model
import xgboost as xgb
from validmind.datasets.classification import customer_churn as demo_dataset

df = demo_dataset.load_data()

vm_dataset = vm.init_dataset(
    dataset=df,
    target_column=demo_dataset.target_column,
    class_labels=demo_dataset.class_labels
)

train_df, validation_df, test_df = demo_dataset.preprocess(df)

x_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
x_val = validation_df.drop(demo_dataset.target_column, axis=1)
y_val = validation_df[demo_dataset.target_column]

model = xgb.XGBClassifier(early_stopping_rounds=10)
model.set_params(
    eval_metric=["error", "logloss", "auc"],
)
model.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=False,
)

In [ ]:
vm_train_ds = vm.init_dataset(
    dataset=train_df,
    type="generic",
    target_column=demo_dataset.target_column
)

vm_test_ds = vm.init_dataset(
    dataset=test_df,
    type="generic",
    target_column=demo_dataset.target_column
)

vm_model = vm.init_model(
    model,
    train_ds=vm_train_ds,
    test_ds=vm_test_ds,
)

Now we can run this project's documentation template by passing the required `dataset` and `model` context:

In [ ]:
model_suite = vm.run_documentation_tests(model=vm_model, dataset=vm_dataset)